<a href="https://colab.research.google.com/github/wesberg05/Mental-Health-AI-Model/blob/main/MentalhealthAIModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Define the model name and load the tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Print out which device we're using (GPU or CPU)
print(device)

cuda


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")

# Split the dataset into training and evaluation sets
dataset_split = dataset['train'].train_test_split(test_size=0.1)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

In [ ]:
# Check the column names of the dataset
print(dataset.column_names)

{'train': ['Context', 'Response']}


In [ ]:
def preprocess_function(examples):
    # Use 'Context' as the input and 'Response' as the target
    inputs = [doc for doc in examples['Context']]
    targets = [doc for doc in examples['Response']]

    # Tokenize the input conversations (context)
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt")

    # Tokenize the responses (targets)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, padding="max_length", truncation=True, return_tensors="pt")

    # Attach the tokenized responses as labels
    model_inputs["labels"] = labels["input_ids"]

    # Move the tokenized inputs and labels to the appropriate device (GPU/CPU)
    model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

    return model_inputs

# Tokenize the training and evaluation datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainingArguments

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',              # Directory to save the model checkpoints
    evaluation_strategy="epoch",         # Evaluate at the end of every epoch
    learning_rate=2e-5,                  # Learning rate for the optimizer
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=8,        # Batch size for evaluation
    weight_decay=0.01,                   # Add weight decay to prevent overfitting
    save_total_limit=3,                  # Only keep the last 3 checkpoints
    num_train_epochs=10,                  # Number of epochs (iterations over the dataset)
    predict_with_generate=True,          # Enable text generation during evaluation
    logging_dir="./logs",                # Directory to store training logs
    logging_steps=100,                   # Log every 100 steps
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,                           # The pre-trained model (FLAN-T5)
    args=training_args,                    # Training arguments defined above
    train_dataset=tokenized_train_dataset, # Tokenized training dataset
    eval_dataset=tokenized_eval_dataset,   # Tokenized evaluation dataset
    tokenizer=tokenizer                    # Tokenizer used for tokenization
)

In [ ]:
# Start the training process
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.917600,3.336186
2,3.434300,2.981055
3,3.172600,2.899305
4,3.103200,2.870415
5,3.076900,2.852989
6,3.056800,2.840460
7,3.007700,2.832532
8,3.073400,2.827158
9,3.037500,2.824168
10,3.017700,2.823113


TrainOutput(global_step=3950, training_loss=3.2708146377756626, metrics={'train_runtime': 1701.5038, 'train_samples_per_second': 18.572, 'train_steps_per_second': 2.321, 'total_flos': 5874139948646400.0, 'train_loss': 3.2708146377756626, 'epoch': 10.0})

In [ ]:
def generate_response(context):
    # Tokenize the input context
    inputs = tokenizer(context, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate a response with more flexibility in length
    response_ids = model.generate(
        inputs["input_ids"],
        max_length=350,            # Increase max_length for longer responses
        min_length=125,             # Set a minimum length to ensure responses are substantive
        num_beams=8,               # Increase number of beams for diversity
        early_stopping=True,        # Allow early stopping if the response is complete
        no_repeat_ngram_size=2,    # Prevent repetition of phrases
        temperature=0.9,           # Introduce some randomness to the generation
        top_k=50,                  # Limit to the top 50 choices
        top_p=0.95                 # Nucleus sampling for diverse output
    )

    # Decode the generated response and return it
    return tokenizer.decode(response_ids[0], skip_special_tokens=True)

# Test the model with a sample input context
sample_context = """
Client: I’ve been feeling really anxious about my job lately. My workload has increased significantly, and I feel like I’m always on edge, worrying that I might not meet my deadlines. I also find it hard to concentrate, and sometimes I even feel physically sick when I think about work. I want to perform well, but this pressure is overwhelming. How can I manage this anxiety and find a better work-life balance?

"""

# Generate a response from the model
generated_response = generate_response(sample_context)
print("Generated Response:", generated_response)

Generated Response: I'm sorry to hear that you've been feeling anxious about your job. It's a great feeling to be able to focus on what you are doing and what is going on in your life. There are many reasons why you feel anxious, but you can't be sure if you have the courage to do so. You're not alone. If you want to work on your own, you may need to have some time to think about how you would like to improve your work-life balance. The best way to manage this anxiety is to try to figure out how to deal with it.


In [ ]:
import huggingface_hub

# Publish the model
from google.colab import userdata

REPO_NAME = "MentalHealthChatbot"
HF_TOKEN = userdata.get("HF_TOKEN")

# save model and tokenizer
model.save_pretrained(REPO_NAME)
tokenizer.save_pretrained(REPO_NAME)

# push model and tokenizer to huggingface
model.push_to_hub(REPO_NAME, token=HF_TOKEN)
tokenizer.push_to_hub(REPO_NAME, token=HF_TOKEN)

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/wesberg05/MentalHealthChatbot/commit/54d03fa08010ef980db76206cf7abe6750643e99', commit_message='Upload tokenizer', commit_description='', oid='54d03fa08010ef980db76206cf7abe6750643e99', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Import the required modules
from transformers import pipeline

# Check if a GPU is available
import torch
device = 0 if torch.cuda.is_available() else -1

# Load the fine-tuned model from Hugging Face Hub
model = pipeline("text2text-generation", model="wesberg05/MentalHealthChatbot", device=device)

print("Environment set up. Model loaded on:", "GPU" if device == 0 else "CPU")

def generate_response(context):
    # Generate a response from the model with specified constraints
    response = model(context,
                     max_new_tokens=350,       # Adjust as needed for response length
                     min_length=125,            # Minimum length of the response
                     num_beams=8,              # Number of beams for diversity
                     early_stopping=True,       # Stop early if the response is complete
                     no_repeat_ngram_size=2,    # Prevent repetition of phrases
                     do_sample=True,           # Enable sampling
                     temperature=0.9,          # Introduce randomness
                     top_k=50,                 # Limit to top 50 choices
                     top_p=0.95                # Nucleus sampling for diverse output
    )

    # Return the generated response text
    return response[0]['generated_text']

# Test the model with a sample input context
sample_context = """
Client: I’ve been feeling really anxious about my job lately. My workload has increased significantly, and I feel like I’m always on edge, worrying that I might not meet my deadlines. I also find it hard to concentrate, and sometimes I even feel physically sick when I think about work. I want to perform well, but this pressure is overwhelming. How can I manage this anxiety and find a better work-life balance?
"""

# Generate a response from the model
generated_response = generate_response(sample_context)
print("Generated Response:", generated_response)

Environment set up. Model loaded on: GPU
Generated Response: I'm sorry to hear that you've been feeling anxious about your job. It sounds like you have a lot of work to do, so it's important to make sure you are able to manage your anxiety and focus on your work. There are many ways you can manage this anxiety as well. If you don't have time to think about it, it may be helpful to talk to your supervisor about how you feel. You can also talk with your boss about what is going on in your life, and ask if he/she is willing to help you with the anxiety.
